# Experiment Tracking
- 실험을 여러개 할때 그 과정과 결과를 좀 더 편리하게 모으고 파악하기 좋게 만들어 주는 법에 대해서 다룬다
- ML하려면 Hyperparameter 조정해 가면서 이런저런 시도를 해봐야 할 텐데 그때 꽤 유용할 것으로 보인다

### 1. TensorBoard 대충 실행해 보기
- 1-1 : 데이터 다운로드
- 1-2 : Transform 복습 / DataLoader 만들기
- 1-3 : Pretrained model 조정 복습 / summary로 보기 복습
- 1-4 : torch.utils.tensorboard.SummaryWriter를 사용해 Track하도록 train loop 변형
- 1-5 : 다 갖춰졌으니 training 해보고 TensorBoard로 보기
    - VSCode에서 Ctrl + Shift + P -> Python: Launch TensorBoard 로 볼수 있음

### 2. 여러개 실행하고 실행한거 트랙해 보기
- 2-1 : SummaryWriter 생성 도구 (대략 어느 폴더에 어떻게 잘 눈에 띄게 저장할까)
- 2-2 : Data download & Transforms 준비 & DataLoader 준비
    - 원래 EfficientB0와 EfficientB2는 서로 다른 transform을 갖고 있는데 책에서는 그냥 같은 transform 적용. 일단 그거 그대로 따라가긴 했는데 진짜로 할때는 그렇게 하면 안되겠지
- 2-3 : Model 생성 도구
- 2-4 : 여러 다른 조건에서 테스트 후 TensorBoard로 보기
    - 근데 막 엄청 차이나지는 않음 역시...

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cuda


In [2]:
# 1-1 : 데이터 다운로드

from going_modular import *

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
print(f'image_path: {image_path}')

[INFO] data/pizza_steak_sushi directory exists, skipping download.
image_path: data/pizza_steak_sushi


In [3]:
# 1-2 : Transform 복습 / DataLoader 만들기

# Create DataLoaders
import torchvision

train_dir = image_path / 'train'
test_dir = image_path / 'test'

# Manual version
manual_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize( mean = [0.485, 0.456, 0.406],  
                                       std = [0.229, 0.224, 0.225])
])
print(f'manual_transforms: {manual_transforms}')

# Automatic version
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print(f'automatic_transforms: {automatic_transforms}')

# We use automatic version
train_dataloader, test_dataloader, class_names = create_dataloaders(
                                                    train_dir=train_dir,
                                                    test_dir=test_dir,
                                                    transform=automatic_transforms,
                                                    batch_size=32
                                                )
print(f'len(train_dataloader.dataset): {len(train_dataloader.dataset)}')
print(f'len(test_dataloader.dataset): {len(test_dataloader.dataset)}')
print(f'class_names: {class_names}')


manual_transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
automatic_transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
len(train_dataloader.dataset): 225
len(test_dataloader.dataset): 75
class_names: ['pizza', 'steak', 'sushi']


In [4]:
# 1-3 : Pretrained model 조정 복습 / summary로 보기 복습

import torchinfo

# Get pretrained model, freeze base layers and change the classifier

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Change the classifier
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=len(class_names),
                    bias=True)
).to(device)

# print summary
torchinfo.summary(
            model, 
            input_size=(32, 3, 224, 224),
            verbose=0,
            col_names=['input_size', 'output_size','num_params', 'trainable'],
            row_settings=['var_names']
        )

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential (features)                                      [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]        [32, 32, 112, 112]        --         

In [5]:
# 1-4 : torch.utils.tensorboard.SummaryWriter를 사용해 Track하도록 train loop 변형

# Define train function that can use SummaryWriter
import torch
import torch.utils.tensorboard

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_v1 (
        model: torch.nn.Module, 
        train_dataloader: torch.utils.data.DataLoader, 
        test_dataloader: torch.utils.data.DataLoader, 
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        epochs: int,
        device: torch.device,
        writer: torch.utils.tensorboard.SummaryWriter
    ) -> Dict[str, List]:
    
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      
    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for 
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]} 
      For example if training for epochs=2: 
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
        # Track the PyTorch model architecture
        writer.add_graph(model=model, 
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    # Close the writer
    writer.close()
    
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

2024-01-15 06:21:55.435605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 06:21:55.472740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 06:21:55.472771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 06:21:55.473558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 06:21:55.479115: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# Define writer, loss and optimizer and then train
from going_modular import *

writer = torch.utils.tensorboard.SummaryWriter()
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

results = train_v1(
            model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_func,
            epochs=5,
            device=device,
            writer=writer
        )

# Ctrl + Shift + P -> Python: Launch TensorBoard 해서 TensorBoard로 보자

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0735 | train_acc: 0.4297 | test_loss: 0.7943 | test_acc: 0.8456
Epoch: 2 | train_loss: 0.8468 | train_acc: 0.7148 | test_loss: 0.7604 | test_acc: 0.7841
Epoch: 3 | train_loss: 0.7873 | train_acc: 0.7266 | test_loss: 0.6967 | test_acc: 0.8551
Epoch: 4 | train_loss: 0.6480 | train_acc: 0.7539 | test_loss: 0.5912 | test_acc: 0.8759
Epoch: 5 | train_loss: 0.5841 | train_acc: 0.8867 | test_loss: 0.5857 | test_acc: 0.8968


In [7]:
# - 2-1 : SummaryWriter 생성 도구 (대략 어느 폴더에 어떻게 잘 눈에 띄게 저장할까)

# Create a helper function to build SummaryWriter instances
import datetime
import os
import torch
import torch.utils.tensorboard

def create_writer(experiment_name: str, 
                  model_name: str, 
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return torch.utils.tensorboard.writer.SummaryWriter(log_dir=log_dir)

In [12]:
# 결론적으로 봤을때 v1이랑 같다. 뭔가 더 들어갔나 싶었는데 결국 바뀐게 없네.

# Define train function that can use SummaryWriter

import torch
import torch.utils.tensorboard

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_v2 (
        model: torch.nn.Module, 
        train_dataloader: torch.utils.data.DataLoader, 
        test_dataloader: torch.utils.data.DataLoader, 
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        epochs: int,
        device: torch.device,
        writer: torch.utils.tensorboard.SummaryWriter
    ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for 
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]} 
      For example if training for epochs=2: 
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
        }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc}, 
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

In [23]:
# - 2-2 : Data download & Transforms 준비 & DataLoader 준비

#
# Multiple experiments
# - 10% data vs 20% data
# - efficient_b0 vs efficient_b2
# - 5 epochs vs 10 epochs
#

data_10_path = download_data(
                source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                destination="pizza_steak_sushi")

train_dir_10 = data_10_path / 'train'
print(f'train_dir_10: {train_dir_10}')

data_20_path = download_data(
                source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                destination="pizza_steak_sushi_20_percent")

train_dir_20 = data_20_path / 'train'
print(f'train_dir_20: {train_dir_20}')

# We use same data for test
test_dir = data_10_path / 'test'
print(f'test_dir: {test_dir}')

# Create dataloaders
# Create common transforms
#
# Note: EfficientB0와 EfficientB2의 DEFAULT transform은 사실 다르다
#       근데 메모리가 많이 들기도 하고 책에서도 그냥 똑같은 크기로 바꿔 버리니까 여기선
#       일단 그렇게 가기로 한다
#

BATCH_SIZE = 32

simple_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize( mean = [0.485, 0.456, 0.406],  
                                       std = [0.229, 0.224, 0.225])
])

train_dataloader_10, test_dataloader, class_names = create_dataloaders(
                                                        train_dir=train_dir_10,
                                                        test_dir=test_dir,
                                                        transform=simple_transform,
                                                        batch_size=BATCH_SIZE
                                                    )

train_dataloader_20, _, _ = create_dataloaders(
                                train_dir=train_dir_20,
                                test_dir=test_dir,
                                transform=simple_transform,
                                batch_size=BATCH_SIZE
                            )

print(f'len(train_dataloader_10.dataset): {len(train_dataloader_10.dataset)}')
print(f'len(train_dataloader_20.dataset): {len(train_dataloader_20.dataset)}')
print(f'len(test_dataloader.dataset): {len(test_dataloader.dataset)}')


[INFO] data/pizza_steak_sushi directory exists, skipping download.
train_dir_10: data/pizza_steak_sushi/train
[INFO] data/pizza_steak_sushi_20_percent directory exists, skipping download.
train_dir_20: data/pizza_steak_sushi_20_percent/train
test_dir: data/pizza_steak_sushi/test
len(train_dataloader_10.dataset): 225
len(train_dataloader_20.dataset): 450
len(test_dataloader.dataset): 75


In [28]:
# - 2-3 : Model 생성 도구

# Create model creation function
import torchinfo

def create_effnetb0(out_features:int):
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)

    # Freeze base layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Change the classifier
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(0.2, inplace=True),
        torch.nn.Linear(in_features=1280,
                        out_features=out_features,
                        bias=True)
    ).to(device)

    model.name = 'effnetb0'
    return model

def create_effnetb2(out_features:int):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    # Freeze base layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Change the classifier
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(0.2, inplace=True),
        torch.nn.Linear(in_features=1408, # torchinfo.summary로 확인
                        out_features=out_features,
                        bias=True)
    ).to(device)

    model.name = 'effnetb2'
    return model

In [26]:
effnetb0 = create_effnetb0(len(class_names))
torchinfo.summary(
            model = effnetb0, 
            input_size=(32, 3, 224, 224), 
            col_names = ["input_size", "output_size", "num_params", "trainable"],
            col_width = 20,
            row_settings = ['var_names']
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [27]:
effnetb2 = create_effnetb2(len(class_names))
torchinfo.summary(
            model = effnetb2, 
            input_size=(32, 3, 224, 224), 
            col_names = ["input_size", "output_size", "num_params", "trainable"],
            col_width = 20,
            row_settings = ['var_names']
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [31]:
- 2-4 : 여러 다른 조건에서 테스트 후 TensorBoard로 보기

# Create experiments and set up training code
# 1. Create epochs list
num_epochs = [5, 10]

# 2. Create models list (need to create a new model for each experiment)
models = ["effnetb0", "effnetb2"]

# 3. Create dataloaders dictionary for various dataloaders
train_dataloaders = {"data_10_percent": train_dataloader_10,
                     "data_20_percent": train_dataloader_20}

# Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each DataLoader
for dataloader_name, train_dataloader in train_dataloaders.items():

    # 4. Loop through each number of epochs
    for epochs in num_epochs: 

        # 5. Loop through each model name and create a new model based on the name
        for model_name in models:

            # 6. Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # 7. Select the model
            if model_name == "effnetb0":
                model = create_effnetb0(len(class_names)) # creates a new model each time (important because we want each experiment to start from scratch)
            else:
                model = create_effnetb2(len(class_names)) # creates a new model each time (important because we want each experiment to start from scratch)
            
            # 8. Create a new loss and optimizer for every model
            loss_fn = torch.nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            # 9. Train target model with target dataloaders and track experiments
            train_v2(
                model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader, 
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=epochs,
                device=device,
                writer=create_writer(experiment_name=dataloader_name,
                                     model_name=model_name,
                                     extra=f"{epochs}_epochs"))
            
            # 10. Save the model to file so we can get back the best model
            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter, saving to: runs/2024-01-15/data_10_percent/effnetb0/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0508 | train_acc: 0.3945 | test_loss: 0.8675 | test_acc: 0.6705
Epoch: 2 | train_loss: 0.9289 | train_acc: 0.6133 | test_loss: 0.6877 | test_acc: 0.8854
Epoch: 3 | train_loss: 0.8018 | train_acc: 0.7148 | test_loss: 0.6782 | test_acc: 0.8561
Epoch: 4 | train_loss: 0.6824 | train_acc: 0.7812 | test_loss: 0.6827 | test_acc: 0.8258
Epoch: 5 | train_loss: 0.6040 | train_acc: 0.9023 | test_loss: 0.6434 | test_acc: 0.8665
[INFO] Saving model to: models/07_effnetb0_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb2
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter, saving to: runs/2024-01-15/data_10_percent/effnetb2/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0354 | train_acc: 0.3828 | test_loss: 0.9575 | test_acc: 0.6203
Epoch: 2 | train_loss: 0.9243 | train_acc: 0.6094 | test_loss: 0.7848 | test_acc: 0.8864
Epoch: 3 | train_loss: 0.7670 | train_acc: 0.7578 | test_loss: 0.7605 | test_acc: 0.7538
Epoch: 4 | train_loss: 0.6949 | train_acc: 0.9062 | test_loss: 0.6795 | test_acc: 0.8968
Epoch: 5 | train_loss: 0.6208 | train_acc: 0.8789 | test_loss: 0.6831 | test_acc: 0.8665
[INFO] Saving model to: models/07_effnetb2_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 3
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2024-01-15/data_10_percent/effnetb0/10_epochs...


...

Epoch: 1 | train_loss: 0.9404 | train_acc: 0.5938 | test_loss: 0.7761 | test_acc: 0.8561
Epoch: 2 | train_loss: 0.6945 | train_acc: 0.8271 | test_loss: 0.6328 | test_acc: 0.8968
Epoch: 3 | train_loss: 0.5672 | train_acc: 0.8479 | test_loss: 0.5496 | test_acc: 0.9176
Epoch: 4 | train_loss: 0.5010 | train_acc: 0.8688 | test_loss: 0.5615 | test_acc: 0.8873
Epoch: 5 | train_loss: 0.4411 | train_acc: 0.8646 | test_loss: 0.4853 | test_acc: 0.8873
[INFO] Saving model to: models/07_effnetb2_data_20_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 7
[INFO] Model: effnetb0
[INFO] DataLoader: data_20_percent
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2024-01-15/data_20_percent/effnetb0/10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9932 | train_acc: 0.4917 | test_loss: 0.7131 | test_acc: 0.8665
Epoch: 2 | train_loss: 0.7070 | train_acc: 0.8313 | test_loss: 0.5754 | test_acc: 0.8968
Epoch: 3 | train_loss: 0.5647 | train_acc: 0.8771 | test_loss: 0.4591 | test_acc: 0.8968
Epoch: 4 | train_loss: 0.5577 | train_acc: 0.8479 | test_loss: 0.4023 | test_acc: 0.9072
Epoch: 5 | train_loss: 0.4622 | train_acc: 0.8625 | test_loss: 0.3850 | test_acc: 0.9176
Epoch: 6 | train_loss: 0.4233 | train_acc: 0.8875 | test_loss: 0.3563 | test_acc: 0.9072
Epoch: 7 | train_loss: 0.4194 | train_acc: 0.8688 | test_loss: 0.3299 | test_acc: 0.9489
Epoch: 8 | train_loss: 0.4367 | train_acc: 0.8646 | test_loss: 0.2972 | test_acc: 0.8968
Epoch: 9 | train_loss: 0.3188 | train_acc: 0.9271 | test_loss: 0.3288 | test_acc: 0.9280
Epoch: 10 | train_loss: 0.3019 | train_acc: 0.9271 | test_loss: 0.2712 | test_acc: 0.9280
[INFO] Saving model to: models/07_effnetb0_data_20_percent_10_epochs.pth
------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0049 | train_acc: 0.5062 | test_loss: 0.7858 | test_acc: 0.8968
Epoch: 2 | train_loss: 0.7413 | train_acc: 0.8021 | test_loss: 0.6790 | test_acc: 0.8769
Epoch: 3 | train_loss: 0.6294 | train_acc: 0.8396 | test_loss: 0.5692 | test_acc: 0.9176
Epoch: 4 | train_loss: 0.4857 | train_acc: 0.8854 | test_loss: 0.5377 | test_acc: 0.9176
Epoch: 5 | train_loss: 0.4874 | train_acc: 0.8688 | test_loss: 0.4784 | test_acc: 0.9280
Epoch: 6 | train_loss: 0.4321 | train_acc: 0.8938 | test_loss: 0.4305 | test_acc: 0.9176
Epoch: 7 | train_loss: 0.3883 | train_acc: 0.8896 | test_loss: 0.4567 | test_acc: 0.9176
Epoch: 8 | train_loss: 0.3447 | train_acc: 0.9229 | test_loss: 0.4000 | test_acc: 0.9176
Epoch: 9 | train_loss: 0.3064 | train_acc: 0.9313 | test_loss: 0.4175 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.3440 | train_acc: 0.9250 | test_loss: 0.4031 | test_acc: 0.9176
[INFO] Saving model to: models/07_effnetb2_data_20_percent_10_epochs.pth
------------------------------------